In [2]:
### import necessary libraries
import os
import ee
import sys
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
from pprint import pprint
import matplotlib.pyplot as plt

# Sube un nivel en la estructura de directorios para llegar a la raíz del proyecto
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Ahora puedes importar la función refined_lee
from utils.refined_lee import refined_lee

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [3]:
# Import the MODIS/061/MOD13Q1 image collection
modis = ee.ImageCollection("MODIS/061/MOD13Q1")

# Define the time of interest
start_year = 2010
end_year = 2024
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year, 12, 31)

filtered = modis\
    .filterDate(start_date, end_date)\

# Get the 1 image from the collection
image = filtered.first()

Map = geemap.Map()

def bitwise_extract(input, from_bit, to_bit):
    mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
    mask = ee.Number(1).leftShift(mask_size).subtract(1)
    return input.rightShift(from_bit).bitwiseAnd(mask)

def mask_snow_and_clouds(image):
    summary_qa = image.select('SummaryQA')
    # Select pixels which are less than or equals to 1 (0 or 1)
    qa_mask = bitwise_extract(summary_qa, 0, 1).lte(1)
    masked_image = image.updateMask(qa_mask)
    return masked_image.copyProperties(image, ['system:index', 'system:time_start'])

# Mask the image and select the 'NDVI' band before visualizing
image_masked = ee.Image(mask_snow_and_clouds(image)).select('NDVI')

masked_col = filtered.map(mask_snow_and_clouds)
ndvi_col = masked_col.select('NDVI')

# Función para escalar los valores NDVI y copiar las propiedades deseadas
def scale_ndvi(image):
    scaled = image.divide(10000)
    return scaled.copyProperties(source=image, properties=['system:index', 'system:time_start'])

# Aplicar la función a cada imagen en la colección
ndvi_scaled = ndvi_col.map(scale_ndvi)

# Extract 1 image from the ndv_scaled collection
# Visualize it and add it into the map

# Add NDVI to map
# Map.addLayer(image.select('NDVI'), vis_params, 'MODIS NDVI')

# map.addLayer(ndvi_scaled, vis_params_2, 'MODIS NDVI (Masked & Scaled)')

# Filter the collection by date
filtered = modis.filter(ee.Filter.date(start_date, end_date))

def bitwise_extract(input, from_bit, to_bit):
    """
    Masks bits of an image between specified bit positions.
    """
    mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
    mask = ee.Number(1).leftShift(mask_size).subtract(1)
    return input.rightShift(from_bit).bitwiseAnd(mask)

def mask_snow_and_clouds(image):
    """
    Masks snow and clouds based on the SummaryQA band.
    """
    summary_qa = image.select('SummaryQA')
    qa_mask = bitwise_extract(summary_qa, 0, 1).lte(1)
    masked_image = image.updateMask(qa_mask)
    return masked_image.copyProperties(image, ['system:index', 'system:time_start'])

# Apply cloud and snow masking
masked_col = filtered.map(mask_snow_and_clouds)

ndvi_col = masked_col.select('NDVI')

def scale_ndvi(image):
    """
    Scales NDVI values from MODIS.
    """
    scaled = image.divide(10000)
    return scaled.copyProperties(image, ['system:index', 'system:time_start'])

# Scale NDVI values
ndvi_scaled = ndvi_col.map(scale_ndvi)

In [4]:
years = ee.List.sequence(start_year, end_year)
months = ee.List.sequence(1, 12)

def calculate_monthly_images(years, months, ndvi_scaled):
    monthly_images = []
    for y in years.getInfo():
        for m in months.getInfo():
            image = ndvi_scaled \
                .filter(ee.Filter.calendarRange(y, y, 'year')) \
                .filter(ee.Filter.calendarRange(m, m, 'month')).mean() \
                .set({
                    'year': y,
                    'month': m,
                    'system:time_start': ee.Date.fromYMD(y, m, 1).millis()
                })
            monthly_images.append(image)
    return monthly_images

monthly_images = calculate_monthly_images(years, months, ndvi_scaled)


# Create a collection

monthly_images_col = ee.ImageCollection.fromImages(monthly_images)

# Filter the collection for the image for May 2015 and display the image on the map
image_2015_05 = monthly_images_col.filter(ee.Filter.eq('year', 2015)).filter(ee.Filter.eq('month', 5)).first()

print(monthly_images_col.size().getInfo()) 

180


In [36]:
# Imprimir el número de imágenes en la colección
print(monthly_images_col.size().getInfo())

180


In [5]:
# Crear una lista de meses
months = list(range(1, 13))

# Función para calcular el NDVI mínimo para un mes dado
def min_ndvi_for_month(m):
    # Filtrar la colección de imágenes por mes
    monthly_images = monthly_images_col.filter(ee.Filter.calendarRange(m, m, 'month'))
    # Mask values equal or less than 0
    monthly_images = monthly_images.map(lambda img: img.updateMask(img.gt(0)))
    # Calcular el NDVI mínimo
    min_ndvi = monthly_images.reduce(ee.Reducer.min())
    # Devolver la imagen con NDVI mínimo
    return min_ndvi.set('month', m)

# Calcular el NDVI mínimo para cada mes
monthly_min_images = [min_ndvi_for_month(m) for m in months]

# Imprimir el tamaño de la lista de imágenes
print(len(monthly_min_images))

# Set visualization parameters.
vis_params_min = {
    'min': 0,  # Ajustado basado en el rango de salida
    'max': 0.8,  # Ajustado basado en el rango de salida
    'palette': ['white', 'blue']
}

# Add it to the map
Map.addLayer(monthly_min_images[0], vis_params_min, 'Min NDVI - Jan')

12


In [6]:
# Crear una lista de meses
months = list(range(1, 13))

# Función para calcular el NDVI máximo para un mes dado
def max_ndvi_for_month(m):
    # Filtrar la colección de imágenes por mes
    monthly_images = monthly_images_col.filter(ee.Filter.calendarRange(m, m, 'month'))
    # Mask values equal or less than 0
    monthly_images = monthly_images.map(lambda img: img.updateMask(img.gt(0)))
    # Calcular el NDVI máximo
    max_ndvi = monthly_images.reduce(ee.Reducer.max())
    # Devolver la imagen con NDVI máximo
    return max_ndvi.set('month', m)

# Calcular el NDVI máximo para cada mes
monthly_max_images = [max_ndvi_for_month(m) for m in months]

# Imprimir el tamaño de la lista de imágenes
print(len(monthly_max_images))
# Imprimir el valor máximo por cada mes
pprint(monthly_max_images)

# Set visualization parameters.
vis_params_max = {
    'min': 0,  # Ajustado basado en el rango de salida
    'max': 0.8,  # Ajustado basado en el rango de salida
    'palette': ['white', 'red']
}

# Add it to the map
Map.addLayer(monthly_max_images[0], vis_params_max, 'Max NDVI - Jan')

12


In [8]:
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…